<a href="https://colab.research.google.com/github/EddyGiusepe/Pytorch_Deep_Learning/blob/main/Classification_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align='center'>Classificação binária, Breast Cancer, com Pytorch</h2>


Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Este Script está baseado no seguinte vídeo: [Classificação Binária com Pytorch](https://www.youtube.com/watch?v=cZf3CMdUKIY&t=38s).

## Importação das bibliotecas

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
torch.__version__


'1.10.0+cu111'

## Base de Dados

In [3]:
np.random.seed(123)

torch.manual_seed(123)

In [4]:
Dataset = pd.read_csv('/content/drive/MyDrive/2_DEEP_LEARNING_REDES_NEURAIS_Jorge/1_Pytorch_Deep_Learning/Pytorch_examples/data_breast_cancer.csv')

Dataset.head(6)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
5,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,NaN


In [5]:
# Criando a classe

# creating instance of labelencoder
labelencoder = LabelEncoder()

classe = pd.DataFrame(labelencoder.fit_transform(Dataset['diagnosis']))
classe.sample(6)

,0
333,0
273,0
201,1
178,0
85,1
500,0


In [6]:
# criando os previsores

previsores = Dataset.drop(columns=['id', 'diagnosis', 'Unnamed: 32'])
# previsores = Dataset.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1, inplace=True) --> Assim não funcionou não


In [7]:
previsores.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
previsores.shape

(569, 30)

In [9]:
# Convertemos para TENSORES 
previsores = torch.tensor(np.array(previsores), dtype=torch.float)

classe = torch.tensor(np.array(classe), dtype=torch.float)


In [10]:
# Verificamos o TENSOR
previsores.shape

torch.Size([569, 30])

In [11]:
# Também
type(previsores)

torch.Tensor

In [12]:
previsores

tensor([[1.7990e+01, 1.0380e+01, 1.2280e+02,  ..., 2.6540e-01, 4.6010e-01,
         1.1890e-01],
        [2.0570e+01, 1.7770e+01, 1.3290e+02,  ..., 1.8600e-01, 2.7500e-01,
         8.9020e-02],
        [1.9690e+01, 2.1250e+01, 1.3000e+02,  ..., 2.4300e-01, 3.6130e-01,
         8.7580e-02],
        ...,
        [1.6600e+01, 2.8080e+01, 1.0830e+02,  ..., 1.4180e-01, 2.2180e-01,
         7.8200e-02],
        [2.0600e+01, 2.9330e+01, 1.4010e+02,  ..., 2.6500e-01, 4.0870e-01,
         1.2400e-01],
        [7.7600e+00, 2.4540e+01, 4.7920e+01,  ..., 0.0000e+00, 2.8710e-01,
         7.0390e-02]])

## Transformação dos Dados para Tensores

In [13]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe),
                                           batch_size=10, shuffle=True) 


In [14]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

## Construção do Modelo

In [15]:
class classificador_torch(nn.Module):

  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(30, 8) # Camada de entrada 
    
    # Mesmos parâmetros usados pelo Kernel initializer do Keras, https://keras.io/api/layers/initializers/ (ver sessão RandomNormal)
    torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)
    self.dense1 = nn.Linear(8, 8) # Camada oculta
    torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)
    self.dense2 = nn.Linear(8, 1) # Camada de saída

    # https://iaexpert.academy/2020/05/25/funcoes-de-ativacao-definicao-caracteristicas-e-quando-usar-cada-uma/#:~:text=ReLU%20%C3%A9%20uma%20abrevia%C3%A7%C3%A3o%20para,n%C3%A3o%20%C3%A9%20centrada%20em%20zero.
    self.activation = nn.ReLU()
    self.dropout = nn.Dropout(0.2)
    self.output =  nn.Sigmoid()

  def forward(self, x): # Aqui fazemos a ligação de todas as camadas
    x = self.dense0(x)
    x = self.activation(x)
    x = self.dropout(x)

    x = self.dense1(x)
    x = self.activation(x)
    x = self.dropout(x)
    
    x = self.dense2(x)
    x = self.output(x)

    return x


  

In [16]:
# Instanciamos nosso Classificador

classificador = classificador_torch()


In [17]:
# Estes foram escolhidos como os melhores... lá no GridSearch

criterion = nn.BCELoss() # Binary Cross Entropy

optimizer = torch.optim.Adam(classificador.parameters(), lr=0.001, weight_decay=0.0001)


## Treinamento do Modelo

In [18]:
for epoch in range(100):
  
  running_loss = 0.

  for data in train_loader:
    inputs, labels = data
    optimizer.zero_grad() # A cada repetição zeramos os gradientes

    outputs = classificador(inputs)
    loss = criterion(outputs, labels) # Aqui calcula o erro entre previsões (outputs) e valores reais (labels)
    loss.backward() # Backpropagation
    optimizer.step() # Aqui se faz a atualização dos pessos


    running_loss += loss.item() # Aqui fazemos a acumulação do erro

  print("Época %3d: perda %.5f" % (epoch + 1, running_loss/len(train_loader)))

# O objetivo é diminuir o valor do erro

Época   1: perda 0.80943
Época   2: perda 0.59553
Época   3: perda 0.52935
Época   4: perda 0.46409
Época   5: perda 0.40749
Época   6: perda 0.36988
Época   7: perda 0.36225
Época   8: perda 0.35697
Época   9: perda 0.31395
Época  10: perda 0.30882
Época  11: perda 0.32959
Época  12: perda 0.31030
Época  13: perda 0.32479
Época  14: perda 0.28704
Época  15: perda 0.29489
Época  16: perda 0.28708
Época  17: perda 0.27333
Época  18: perda 0.28836
Época  19: perda 0.27818
Época  20: perda 0.29999
Época  21: perda 0.25090
Época  22: perda 0.24037
Época  23: perda 0.25740
Época  24: perda 0.28473
Época  25: perda 0.23825
Época  26: perda 0.24456
Época  27: perda 0.23883
Época  28: perda 0.22457
Época  29: perda 0.24069
Época  30: perda 0.22416
Época  31: perda 0.23415
Época  32: perda 0.24657
Época  33: perda 0.22031
Época  34: perda 0.21913
Época  35: perda 0.21345
Época  36: perda 0.20784
Época  37: perda 0.23652
Época  38: perda 0.21905
Época  39: perda 0.21646
Época  40: perda 0.21096


## Classificar somente um registro

In [19]:

novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178, 0.20,
                      0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015, 0.03,
                      0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185, 0.84, 158, 0.363]], dtype = torch.float)

In [20]:
# Para fazer essa previssão, precisamos colocar nosso CLASSIFICADOR em modo de AVALIAÇÃO

classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [21]:
previsao = classificador(novo)

In [22]:
previsao

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [23]:
# Para visualizar os Dados fazemos o seguinte

previsao = previsao.detach()

In [24]:
previsao

tensor([[1.]])

In [25]:
# Passamos para ARRAY, assim:

previsao = previsao.numpy()

In [26]:
type(previsao)

numpy.ndarray

In [27]:
# Percebemos que a previsão deu 1 --> Isso significa que a pessoa tem CÂNCER

previsao

array([[1.]], dtype=float32)

In [28]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

## Salvamos o nosso Classificador

In [29]:
# .state_dict --> nos dá informação das camadas com os pesos

classificador.state_dict

<bound method Module.state_dict of classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)>

In [30]:
# Aqui, mais explícito

classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-3.9721e-01,  1.0573e-01, -1.4631e-01, -3.0919e-02,  2.1050e-02,
                        8.3734e-01,  1.5699e+00,  1.0174e+00, -3.3016e-02, -7.7140e-02,
                       -1.4872e-01,  3.2110e-02,  2.2688e-01,  1.3394e-01,  6.1822e-02,
                        4.6633e-01,  6.2867e-01,  2.0660e-01,  1.2512e-01,  3.6311e-02,
                       -3.3149e-01,  1.2870e-01, -1.5122e-01,  6.7956e-02,  1.1234e-01,
                        1.2849e+00,  1.6499e+00,  1.0370e+00,  2.7712e-01,  2.2880e-01],
                      [ 4.4252e-03,  8.0927e-03,  5.7704e-02, -9.8896e-03, -1.4061e-39,
                       -1.4140e-39,  4.0617e-39, -2.1974e-40, -2.9822e-39, -7.9458e-40,
                       -1.6107e-39,  8.4073e-08, -1.8189e-07,  7.2268e-03, -2.9817e-39,
                        1.0227e-39, -1.4078e-39, -4.3381e-39,  3.4746e-39,  2.6412e-39,
                       -1.3974e-02,  2.6148e-02, -1.9209e-02, -1.5789e-02, -2.4178e-39,


In [31]:
# Agora sim procedemos a SALVAR nosso classificador

torch.save(classificador.state_dict,
           '/content/drive/MyDrive/2_DEEP_LEARNING_REDES_NEURAIS_Jorge/1_Pytorch_Deep_Learning/Pytorch_examples/classificador.pth')